# 第二章 语言模型，提问范式与 Token

 - [一、环境配置](#一、环境配置)
     - [1.1 加载 API key 和一些 Python 的库。](#1.1-加载-API-key-和一些-Python-的库。)
     - [1.2 Helper function 辅助函数](#1.2-Helper-function-辅助函数)
 - [二、尝试向模型提问并得到结果](#二、尝试向模型提问并得到结果)
 - [三、Tokens](#三、Tokens)
 - [四、Helper function 辅助函数 (提问范式)](#四、Helper-function-辅助函数-(提问范式))


在本章中，我们将和您分享大型语言模型（LLM）的工作原理、训练方式以及分词器（tokenizer）等细节对 LLM 输出的影响。我们还将介绍 LLM 的提问范式（chat format），这是一种指定系统消息（system message）和用户消息（user message）的方式，让您了解如何利用这种能力。

## 一、环境配置

### 1.1 加载 API key 和一些 Python 的库。
在本课程中，为您提供了一些加载 OpenAI API key 的代码。

In [ ]:
!pip install openai
!pip install langchain
!pip install --upgrade tiktoken

In [1]:
import os
import openai
# import tiktoken 这个后面没用到，若您对其用处感兴趣，可以参考本文以了解相关内容：https://zhuanlan.zhihu.com/p/629776230

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # 读取本地的.env环境文件。（推荐后续使用这种方法，将 key 放在 .env 文件里。保护自己的 key）

openai.api_key  = 'sk-***' # 更换成您自己的key

### 1.2 Helper function 辅助函数
如果之前曾参加过《ChatGPT Prompt Engineering for Developers》课程，那么对此就相对较为熟悉。
调用该函数输入 Prompt 其将会给出对应的 Completion 。

In [2]:
# 官方文档写法 https://platform.openai.com/overview

def get_completion(prompt, model="gpt-3.5-turbo"):
    """
    使用 OpenAI 的模型生成聊天回复。

    参数:
    prompt: 用户的输入，即聊天的提示。
    model: 使用的模型，默认为"gpt-3.5-turbo"。
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"] # 模型生成的回复

## 二、尝试向模型提问并得到结果

In [13]:
response = get_completion("What is the capital of China?")
print(response)

The capital of China is Beijing.


In [14]:
response = get_completion("中国的首都是哪里？")
print(response)

中国的首都是北京。


## 三、Tokens

In [15]:
# 为了更好展示效果，这里就没有翻译成中文的 Prompt
# 注意这里的字母翻转出现了错误，吴恩达老师正是通过这个例子来解释 token 的计算方式
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

The reversed letters of "lollipop" are "pillipol".


"lollipop" in reverse should be "popillol"

In [17]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

print(response)

p-o-p-i-l-l-o-l


![Tokens.png](../../figures/Tokens.png)

对于英文输入，一个 token 一般对应 4 个字符或者四分之三个单词；对于中文输入，一个 token 一般对应一个或半个词。

不同模型有不同的 token 限制，需要注意的是，这里的 token 限制是输入的 Prompt 和输出的 completion 的 token 数之和，因此输入的 Prompt 越长，能输出的 completion 的上限就越低。

ChatGPT3.5-turbo 的 token 上限是 4096。

## 四、Helper function 辅助函数 (提问范式)
下面是课程中用到的辅助函数。
下图是 OpenAI 提供的一种提问范式，接下来吴恩达老师就是在演示如何利用这种范式进行更好的提问
![Chat-format.png](../../figures/Chat-format.png)

System 信息用于指定模型的规则，例如设定、回答准则等，而 assistant 信息就是让模型完成的具体指令

In [5]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    '''
    封装一个支持更多参数的自定义访问 OpenAI GPT3.5 的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message["content"]

In [6]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

In a garden so bright, a carrot would sprout,
With a cheery orange hue, without a doubt.
With a leafy green top, it danced in the breeze,
A happy carrot, so eager to please.

It grew in the soil, oh so deep and grand,
Stretching its roots, reaching far and expand.
With a joyful smile, it soaked up the sun,
Growing tall and strong, its journey begun.

Days turned to weeks, as it grew day and night,
Round and plump, it was quite a delight.
With every raindrop that fell from above,
The carrot grew sweeter, spreading more love.

At last, the day came when it was time to eat,
With a grin on my face, I took a seat.
I chopped and I sliced, so grateful, you see,
For this happy carrot, bringing joy to me.

So let us remember, when times may get tough,
A happy carrot's journey, it's enough.
For even in darkness, there's always delight,
Just like a carrot, shining so bright.


In [18]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答。'},    
{'role':'user', 
 'content':'就快乐的小鲸鱼为主题给我写一首短诗'},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

在海洋的深处，有一只小鲸鱼，
她快乐又聪明，灵感从不匮乏。
她游遍五大洲，探索未知的秘密，
用歌声传递喜悦，令人心旷神怡。

她跃出海面，高高的飞翔，
尾巴抽空着水花，像梦幻般的画。
她和海豚一起，跳跃在太阳下，
与海洋中的生命，在欢乐中共舞。

她喜欢和海龟一起，缓缓漫游，
看美丽的珊瑚，和色彩鲜艳的鱼群。
她欢迎每个新朋友，无论大或小，
因为在她眼中，每个人都独特而珍贵。

她知道快乐是如此简单，如此宝贵，
在每个时刻中，她都努力传达幸福的表情。
所以当你感到疲惫，沮丧或者低落，
想起小鲸鱼的快乐，让你心中再次充满鲜活。


In [7]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once upon a time, there was a cheerful carrot named Charlie who always brightened everyone's day with his vibrant orange color and contagious laughter.


In [19]:
# 长度控制
messages =  [  
{'role':'system',
 'content':'你的所有答复只能是一句话'},    
{'role':'user',
 'content':'写一个关于快乐的小鲸鱼的故事'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

在追随波浪的起伏中，小鲸鱼快乐地跳跃着，因为它知道游泳的真正乐趣不仅仅在目的地，而是在于享受整个旅程。


In [8]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

Once upon a time, there was a carrot so happy and bright, it danced and sang from morning till night.


In [20]:
# 以上结合
messages =  [  
{'role':'system',
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答，只回答一句话'},    
{'role':'user',
 'content':'写一个关于快乐的小鲸鱼的故事'},
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

在蓝色的大海里，有一只小鲸鱼，无忧无虑，快乐游泳，一切因快乐而变得光辉。


In [9]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    """
    使用 OpenAI 的 GPT-3 模型生成聊天回复，并返回生成的回复内容以及使用的 token 数量。

    参数:
    messages: 聊天消息列表。
    model: 使用的模型名称。默认为"gpt-3.5-turbo"。
    temperature: 控制生成回复的随机性。值越大，生成的回复越随机。默认为 0。
    max_tokens: 生成回复的最大 token 数量。默认为 500。

    返回:
    content: 生成的回复内容。
    token_dict: 包含'prompt_tokens'、'completion_tokens'和'total_tokens'的字典，分别表示提示的 token 数量、生成的回复的 token 数量和总的 token 数量。
    """
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]
    
    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [24]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)
print(response)

In a garden so bright, with colors so cheery,
There lived a carrot, oh so merry!
With a vibrant orange hue, and a leafy green top,
This happy carrot just couldn't stop.

It danced in the breeze, with a joyful sway,
Spreading happiness throughout the day.
With a smile so wide, and eyes full of glee,
This carrot was as happy as can be.

It loved the sunshine, and the rain's gentle touch,
Growing tall and strong, oh so much!
From the earth it sprouted, reaching for the sky,
A happy carrot, oh my, oh my!

So if you're feeling down, just remember this tale,
Of a carrot so happy, it'll never fail.
Find joy in the little things, and let your heart sing,
Just like that carrot, oh what joy it will bring!


In [22]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答。'},    
{'role':'user', 
 'content':'就快乐的小鲸鱼为主题给我写一首短诗'},  
] 
response, token_dict = get_completion_and_token_count(messages)
print(response)

In [25]:
print(token_dict)

{'prompt_tokens': 37, 'completion_tokens': 173, 'total_tokens': 210}


下一个章中，我们将展示如何利用这些组件来评估客户服务助手的输入。
这将是本课程中构建在线零售商客户服务助手的更完整示例的一部分。